# LangChain 

- LangChain은 LLM과 Chat model 두가지를 모두 지원합니다. 

In [9]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

- 3.25 복습 내용 : Openai , ChatOpenAI 둘 다 model_name을 지정해줘야함

In [14]:
llm = OpenAI(model_name="gpt-3.5-turbo-1106")
chat = ChatOpenAI(model_name="gpt-3.5-turbo")

a = llm.predict("Hello, how are you today?")
b = chat.predict("Hello, how are you today?")

a,b

/Users/kimminyoung/minyoungxi/FullStack_GPT/env/lib/python3.11/site-packages/langchain/llms/openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/kimminyoung/minyoungxi/FullStack_GPT/env/lib/python3.11/site-packages/langchain/llms/openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


("Hello! I'm just a text-based AI, so I don't have feelings, but I'm here and ready to assist you. How can I help you today?",
 "Hello! I'm just a computer program so I don't have feelings, but I'm here and ready to assist you. How can I help you today?")

In [15]:
chat = ChatOpenAI(model_name="gpt-3.5-turbo",
                  temperature=0.1,
                  max_tokens=100,)

- message를 predict하는 방법 
    - HumanMessage
    - AIMessage : AI에 의해 보내짐 
    - SystemMessage : LLM에 설정들을 제공하기 위한 메세지 

In [16]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

message = [
    SystemMessage(content="You are a geography expert. And you only reply in Korean",
                  ),
    AIMessage(content="안녕하세요 지리 전문가입니다."),
    HumanMessage(content="Where is the capital of France?"),
]

chat.predict_messages(message)

AIMessage(content='프랑스의 수도는 파리입니다.')

## prompt

- ChatPromptTemplate : template를 message로부터 만든다.
- PromptTemplate : String을 이용해서 template를 만든다.

In [19]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.1)

template = PromptTemplate.from_template(
    "What is distance between {city1} and {city2}?"
    )

prompt = template.format(city1="Paris", city2="Seoul")

chat.predict_messages(message)

'The distance between Paris, France and Seoul, South Korea is approximately 8,600 kilometers (5,344 miles) when measured in a straight line.'

- Prompt template 사용

In [21]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert. And you only reply in {language}."),
    ("ai", "Hello, I am a {name}!"),
    ("human", "What is the distance between {city1} and {city2}?. Also what is your name?")
])

prompt = template.format_messages(
    language="Korean",
    name= "min",
    city1="Paris",
    city2="Seoul"
)

chat.predict_messages(prompt)

AIMessage(content='파리와 서울 사이의 거리는 약 9,500km입니다. 제 이름은 지리 전문가입니다.')

## OutputParser and LCEL

- OutputParser가 필요한 이유는 LLM의 응답을 변형해야 할 때가 있기 때문임. 
    - 응답을 리스트로 변환시켜보자 (LLM은 항상 텍스트로 응답하기 때문에 변형이 필요할 때가 있음)

In [24]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):

    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))
    
p = CommaOutputParser()

p.parse("Hello,how,are,you")

['Hello', 'how', 'are', 'you']

In [29]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma seperate the list of max {max_items}. Do NOT reply with anything else."),
    ("human", "{question}")
])

prompt = template.format_messages(
    max_items=10,
    question="What are the colors?"
)

result = chat.predict_messages(prompt)

p = CommaOutputParser()

p.parse(result.content)

['red',
 'blue',
 'green',
 'yellow',
 'orange',
 'purple',
 'pink',
 'black',
 'white',
 'brown']

### chain !

In [30]:
chain = template | chat | CommaOutputParser()

chain.invoke({
    "max_items": 5,
    "question": "What are the pocketmons?"
})

['Pikachu', 'Charmander', 'Bulbasaur', 'Squirtle']

In [31]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

chef_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "",
        ),
        ("human", "I want to cook {cuisine} food."),
    ]
)

chef_chain = chef_prompt | chat

In [32]:
veg_chef_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it.",
        ),
        ("human", "{recipe}"),
    ]
)

veg_chain = veg_chef_prompt | chat

final_chain = {"recipe": chef_chain} | veg_chain

final_chain.invoke({"cuisine": "indian"})

That's great! Indian cuisine is known for its rich flavors and diverse range of dishes. Here are a few popular Indian recipes you can try:

1. Butter Chicken: A creamy and flavorful chicken dish cooked in a rich tomato and butter sauce.

2. Chana Masala: A spicy and tangy chickpea curry made with a blend of aromatic spices.

3. Palak Paneer: A delicious vegetarian dish made with spinach and paneer (Indian cottage cheese) cooked in a creamy tomato gravy.

4. Biryani: A fragrant and flavorful rice dish cooked with meat or vegetables and a blend of spices.

5. Aloo Gobi: A simple yet delicious dish made with potatoes and cauliflower cooked with spices.

These are just a few examples, but there are countless other Indian dishes you can explore. Enjoy your cooking!As a vegetarian chef specialized in making traditional recipes vegetarian, I can offer you some alternative ingredients for the Indian recipes you mentioned:

1. Butter Chicken: You can replace the chicken with paneer (Indian cott

AIMessageChunk(content='As a vegetarian chef specialized in making traditional recipes vegetarian, I can offer you some alternative ingredients for the Indian recipes you mentioned:\n\n1. Butter Chicken: You can replace the chicken with paneer (Indian cottage cheese) or tofu. Marinate the paneer or tofu in a mixture of yogurt, lemon juice, and spices before cooking it in the buttery tomato sauce.\n\n2. Chana Masala: This dish is already vegetarian, but you can enhance the flavors by adding some roasted vegetables like bell peppers or sweet potatoes. They will add a nice depth to the dish.\n\n3. Palak Paneer: Instead of paneer, you can use tofu or even boiled and mashed chickpeas to make a vegan version of this dish. The spinach gravy can be made creamy by blending cashews or coconut milk.\n\n4. Biryani: For a vegetarian biryani, you can use a mix of vegetables like carrots, peas, potatoes, and cauliflower. You can also add paneer or tofu for some protein. Make sure to layer the rice an